In [3]:
%run ../ImportsConstantsSparkInit.ipynb
sc.setJobGroup("DA_Hours", "DA_Hours")

In [4]:
%%time
pipeline = "[{ $group : { _id : {$dateToString: { format: '%H', date: '$Start_Time' }}, count:{$sum:1}}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
df = df.groupBy("_id").sum("count")
df = df.orderBy("_id")
df = df.withColumnRenamed("_id","Hour").withColumnRenamed("sum(count)","AccidentCount")
print("Number of Documents: " + str(df.count()))
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)

Number of Documents: 24
+------------------+
|sum(AccidentCount)|
+------------------+
|           2845342|
+------------------+

+----+-------------+
|Hour|AccidentCount|
+----+-------------+
|  00|       104143|
|  01|        89545|
|  02|        81704|
|  03|        75138|
|  04|        70740|
|  05|        60245|
|  06|        59218|
|  07|        55371|
|  08|        52401|
|  09|        71108|
|  10|        99071|
|  11|       127905|
|  12|       133314|
|  13|       118336|
|  14|       105461|
|  15|       110452|
|  16|       131412|
|  17|       151621|
|  18|       182674|
|  19|       209736|
|  20|       220530|
|  21|       212802|
|  22|       183146|
|  23|       139269|
+----+-------------+

Wall time: 6.13 s


In [5]:
df.write.format("mongo").option("collection", "AccidentCountByHour").mode("overwrite").save()